In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from datetime import datetime, date
import pandas as pd
import numpy as np
from app.utils import get_db_engine


In [2]:
from sqlalchemy.sql.schema import CheckConstraint, Column, ForeignKey
# https://docs.sqlalchemy.org/en/14/orm/self_referential.html
from sqlalchemy.orm import relationship, backref, validates, deferred
from sqlalchemy.sql.sqltypes import Date
from sqlalchemy.sql.sqltypes import Integer
from sqlalchemy.sql.sqltypes import String

In [23]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()
from app.utils import db_session
# https://statbel.fgov.be/sites/default/files/files/opendata/REFNIS%20code/TU_COM_REFNIS.xlsx
zip_url = "https://statbel.fgov.be/sites/default/files/files/opendata/REFNIS%20code/TU_COM_REFNIS.zip"

class NIC_Code(Base):
    __tablename__ = 'nic_codes'
    
    nic = Column(String(5), primary_key=True)
    parent_nic = Column(String(5), ForeignKey('nic_codes.nic'), nullable=True)
    level = Column(Integer, nullable=False)
    text_nl = Column(String(255))
    text_fr = Column(String(255))
    text_de = Column(String(255))
    valid_from = Column(Date, primary_key=True)
    valid_till = Column(Date, nullable=True)
    
    children = relationship("NIC_Code",
        lazy='select',                    
        backref=backref('parent', remote_side=[nic])
    )
    
    __table_args__ = (
        CheckConstraint('length(nic) = 5', name='nic_length'),
    )
    
    def __repr__(self):
        return """
            <NIC_Code(level='%s', nic='%s', name='%s', parent='%s')>
            """ % (
            self.level,
            self.nic,
            self.text_nl,
            self.parent_nic
        )

    @validates('nic')
    def validate_nic(self, key, nic) -> str:
        if len(nic) != 5:
            raise ValueError("'nic' should be 5 characters long..")
        return nic
    
    @classmethod
    def get_all(cls):
        with db_session(echo=False) as session:
            local_nic_codes_all = session.query(NIC_Code).all()
            session.close()
            if (len(local_nic_codes_all)):
                return local_nic_codes_all
        # extract
        df = pd.read_csv(zip_url, delimiter="|")
        # transform
        df['CD_REFNIS'] = df['CD_REFNIS'].apply(lambda x: '{0:0>5}'.format(x))
        return df[df['CD_SUP_REFNIS'].apply(lambda x: x if (len(x) == 5) else None)]
        # df['DT_VLDT_START'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').date())
        # datetime.strptime(first_date,'%d/%m/%Y').date()
        
        df["DT_VLDT_START"] = np.where(
            df['DT_VLDT_START'] == '01/01/1970',
            date.min,
            df['DT_VLDT_START'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').date())
        )
        df["DT_VLDT_END"] = np.where(
            df['DT_VLDT_END'] == '31/12/9999',
            date.max,
            df['DT_VLDT_END'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').date())
        )
        
        # list(map(lambda x: datetime.datetime.strptime(x,'%b %d, %Y').strftime('%m/%d/%Y'), old_df['oldDate']))
        # list(map(lambda x: datetime.datetime.strptime(x,'%b %d, %Y').strftime('%m/%d/%Y'), old_df['oldDate']))

        # df["DT_VLDT_START"] = np.where(df['DT_VLDT_START'] == '01/01/1970', date.min, datetime.strptime(df['DT_VLDT_START'], '%d/%m/%y'))
        # df["DT_VLDT_END"] = np.where(df['DT_VLDT_END'] == '31/12/9999', date.max, datetime.strptime(df['DT_VLDT_END'], '%d/%m/%y'))
        # date_time_obj = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')

        # df["DT_VLDT_START"] = pd.to_datetime(df['DT_VLDT_START'], errors = 'coerce')
        # df["DT_VLDT_END"] = pd.to_datetime(df['DT_VLDT_END'], errors = 'coerce')
        # print(df)
        
        return df[df["DT_VLDT_END"] == date.max]
        df.rename(columns={
            "LVL_REFNIS": "level",
            "CD_REFNIS": "nic",
            "CD_SUP_REFNIS": "parent_nic",
            "TX_REFNIS_NL": "text_nl",
            "TX_REFNIS_FR": "text_fr",
            "TX_REFNIS_DE": "text_de",
            "DT_VLDT_START": "valid_from",
            "DT_VLDT_END": "valid_till",
        }, inplace=True)
        # load
        # return df
        list = [
            cls(**kwargs) for kwargs in df.to_dict(orient="records")
        ]
        with db_session(echo=False) as session:
            session.bulk_save_objects(list)
            session.commit()
            session.close()
        return list
        with db_session(echo=False) as session:
            return session.query(NIC_Code).all()

        
        return list

In [24]:
nic_codes_table = NIC_Code.__table__

engine = get_db_engine(echo=False)
nic_codes_table.drop(engine, checkfirst=True)
nic_codes_table.create(engine)

df = NIC_Code.get_all()
df

0        None
1        None
2        None
3       02000
4        None
        ...  
2764    93000
2765    93000
2766    93000
2767    93000
2768    93000
Name: CD_SUP_REFNIS, Length: 2769, dtype: object

In [205]:
df = NIC_Code.get_all()
df

[
             <NIC_Code(level='1', nic='02000', name='Vlaams Gewest', parent='-')>
             ,
 
             <NIC_Code(level='1', nic='03000', name='Waals Gewest', parent='-')>
             ,
 
             <NIC_Code(level='1', nic='04000', name='Brussels Hoofdstedelijk Gewest', parent='-')>
             ,
 
             <NIC_Code(level='2', nic='10000', name='Provincie Antwerpen', parent='02000')>
             ,
 
             <NIC_Code(level='2', nic='20000', name='Provincie Brabant', parent='-')>
             ,
 
             <NIC_Code(level='2', nic='20001', name='Provincie Vlaams-Brabant', parent='02000')>
             ,
 
             <NIC_Code(level='2', nic='20002', name='Provincie Waals-Brabant', parent='03000')>
             ,
 
             <NIC_Code(level='2', nic='30000', name='Provincie West-Vlaanderen', parent='02000')>
             ,
 
             <NIC_Code(level='2', nic='40000', name='Provincie Oost-Vlaanderen', parent='02000')>
             ,
 
             <NI

In [171]:
date_str = '09-19-2018'
print(date_str)

print(type(date_str))


date_object = datetime.strptime(date_str, '%m-%d-%Y').date()
print(type(date_object))
print(date_object)

df = NIC_Code.get_all()
df['DT_VLDT_START'].describe()
first_date = df['DT_VLDT_START'][0]
print(first_date)
print(type(first_date))

len(first_date)
datetime.strptime(first_date,'%d/%m/%Y').date()
# df.describe()

09-19-2018
<class 'str'>
<class 'datetime.date'>
2018-09-19


TypeError: list indices must be integers or slices, not str

In [102]:
df = NIC_Code.get_all()
array = df[df["DT_VLDT_START"] != "01/01/1970"]["DT_VLDT_START"].unique()
list(map(lambda x: datetime.strptime(x.strip(' \t\r\n'),'%d/%m/%y').date(), array))


ValueError: unconverted data remains: 95

In [126]:
df = NIC_Code.get_all()
df["DT_VLDT_START"].apply(lambda x: datetime.strptime(x,'%d/%m/%Y').date())
# df.apply(lambda x: datetime.strptime(x,'%d/%m/%y'), df['DT_VLDT_START'])
# df.apply(lambda x: datetime.strptime(x['DT_VLDT_START'],'%d/%m/%y'), axis=1)
# map(lambda x: datetime.strptime(x,'%d/%m/%y'), df['DT_VLDT_START'])
df

,LVL_REFNIS,CD_REFNIS,CD_SUP_REFNIS,TX_REFNIS_DE,TX_REFNIS_FR,TX_REFNIS_NL,DT_VLDT_START,DT_VLDT_END
0,1,02000,-,Flämische Region,Région flamande,Vlaams Gewest,01/01/1970,31/12/9999
1,1,03000,-,Wallonische Region,Région wallonne,Waals Gewest,01/01/1970,31/12/9999
2,1,04000,-,Region Brüssel-Hauptstadt,Région de Bruxelles-Capitale,Brussels Hoofdstedelijk Gewest,01/01/1970,31/12/9999
3,2,10000,02000,Provinz Antwerpen,Province d'Anvers,Provincie Antwerpen,01/01/1970,31/12/9999
4,2,20000,-,Provinz Brabant,Province de Brabant,Provincie Brabant,01/01/1970,31/12/1994
...,...,...,...,...,...,...,...,...
2764,4,93086,93000,Vodelée,Vodelée,Vodelée,01/01/1970,31/12/1976
2765,4,93087,93000,Vogenée,Vogenée,Vogenée,01/01/1970,31/12/1976
2766,4,93088,93000,Walcourt,Walcourt,Walcourt,01/01/1970,31/12/9999
2767,4,93089,93000,Yves-Gomezée,Yves-Gomezée,Yves-Gomezée,01/01/1970,31/12/1976


In [23]:
df = NIC_Code.get_all()
df["DT_VLDT_START"] = np.where(df['DT_VLDT_START'] == '01/01/1970', pd.NaT, df['DT_VLDT_START'])
df["DT_VLDT_END"] = np.where(df['DT_VLDT_END'] == '31/12/9999', pd.NaT, df['DT_VLDT_END'])
df.CD_REFNIS.unique()
# df["DT_VLDT_START"] = pd.to_datetime(df['DT_VLDT_START'])
# df["DT_VLDT_END"] = pd.to_datetime(df['DT_VLDT_END'])

array(['02000', '03000', '04000', ..., '93088', '93089', '93090'],
      dtype=object)

In [12]:
df["DT_VLDT_START"] = pd.to_datetime(df['DT_VLDT_START'], errors = 'coerce')
df["DT_VLDT_END"] = pd.to_datetime(df['DT_VLDT_END'], errors = 'coerce')

df

,LVL_REFNIS,CD_REFNIS,CD_SUP_REFNIS,TX_REFNIS_DE,TX_REFNIS_FR,TX_REFNIS_NL,DT_VLDT_START,DT_VLDT_END
0,1,02000,-,Flämische Region,Région flamande,Vlaams Gewest,1970-01-01,NaT
1,1,03000,-,Wallonische Region,Région wallonne,Waals Gewest,1970-01-01,NaT
2,1,04000,-,Region Brüssel-Hauptstadt,Région de Bruxelles-Capitale,Brussels Hoofdstedelijk Gewest,1970-01-01,NaT
3,2,10000,02000,Provinz Antwerpen,Province d'Anvers,Provincie Antwerpen,1970-01-01,NaT
4,2,20000,-,Provinz Brabant,Province de Brabant,Provincie Brabant,1970-01-01,1994-12-31
...,...,...,...,...,...,...,...,...
2764,4,93086,93000,Vodelée,Vodelée,Vodelée,1970-01-01,1976-12-31
2765,4,93087,93000,Vogenée,Vogenée,Vogenée,1970-01-01,1976-12-31
2766,4,93088,93000,Walcourt,Walcourt,Walcourt,1970-01-01,NaT
2767,4,93089,93000,Yves-Gomezée,Yves-Gomezée,Yves-Gomezée,1970-01-01,1976-12-31


In [152]:
df = pd.read_csv(zip_url, delimiter="|")
df.head()
df['CD_REFNIS'] = df['CD_REFNIS'].apply(lambda x: '{0:0>5}'.format(x))
df.head()
df[df['DT_VLDT_END'] != "31/12/9999"]
df[df['DT_VLDT_START'] != "01/01/1970"]



,LVL_REFNIS,CD_REFNIS,CD_SUP_REFNIS,TX_REFNIS_DE,TX_REFNIS_FR,TX_REFNIS_NL,DT_VLDT_START,DT_VLDT_END
5,2,20001,02000,Provinz Flämisch-Brabant,Province du Brabant flamand,Provincie Vlaams-Brabant,01/01/1995,31/12/9999
6,2,20002,03000,Provinz Wallonisch-Brabant,Province du Brabant wallon,Provincie Waals-Brabant,01/01/1995,31/12/9999
18,3,23000,20001,Bezirk Halle-Vilvoorde,Arrondissement de Hal-Vilvorde,Arrondissement Halle-Vilvoorde,01/01/1995,31/12/9999
20,3,24000,20001,Bezirk Löwen,Arrondissement de Louvain,Arrondissement Leuven,01/01/1995,31/12/9999
22,3,25000,20002,Bezirk Nivelles,Arrondissement de Nivelles,Arrondissement Nijvel,01/01/1995,31/12/9999
...,...,...,...,...,...,...,...,...
2675,4,92139,92000,Gembloux-sur-Orneau,Gembloux-sur-Orneau,Gembloux-sur-Orneau,01/01/1977,31/12/1979
2676,4,92140,92000,Jemeppe-sur-Sambre,Jemeppe-sur-Sambre,Jemeppe-sur-Sambre,01/01/1977,31/12/9999
2677,4,92141,92000,La Bruyère,La Bruyère,La Bruyère,01/01/1977,31/12/9999
2678,4,92142,92000,Gembloux,Gembloux,Gembloux,01/01/1980,31/12/9999


In [94]:
df['CD_REFNIS_CASTED'] = df['CD_REFNIS'].apply(lambda x: '{0:0>5}'.format(x))

In [95]:
df[df["CD_REFNIS"] == 2000]

,LVL_REFNIS,CD_REFNIS,CD_SUP_REFNIS,TX_REFNIS_DE,TX_REFNIS_FR,TX_REFNIS_NL,DT_VLDT_START,DT_VLDT_END,test,CD_REFNIS_CASTED
0,1,2000,-,Flämische Region,Région flamande,Vlaams Gewest,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...,02000


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2769 entries, 0 to 2768
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LVL_REFNIS        2769 non-null   int64 
 1   CD_REFNIS         2769 non-null   int64 
 2   CD_SUP_REFNIS     2769 non-null   object
 3   TX_REFNIS_DE      2769 non-null   object
 4   TX_REFNIS_FR      2769 non-null   object
 5   TX_REFNIS_NL      2769 non-null   object
 6   DT_VLDT_START     2769 non-null   object
 7   DT_VLDT_END       2769 non-null   object
 8   test              2769 non-null   object
 9   CD_REFNIS_CASTED  2769 non-null   object
dtypes: int64(2), object(8)
memory usage: 216.5+ KB


In [52]:
df.columns

Index(['LVL_REFNIS', 'CD_REFNIS', 'CD_SUP_REFNIS', 'TX_REFNIS_DE',
       'TX_REFNIS_FR', 'TX_REFNIS_NL', 'DT_VLDT_START', 'DT_VLDT_END'],
      dtype='object')

In [53]:
df.LVL_REFNIS.unique()

array([1, 2, 3, 4])

In [54]:
df[df["LVL_REFNIS"] == 2]
df[(df["CD_REFNIS"].astype(str).str.startswith("2")) & (df["CD_REFNIS"].astype(str).str.len() == 5)]

,LVL_REFNIS,CD_REFNIS,CD_SUP_REFNIS,TX_REFNIS_DE,TX_REFNIS_FR,TX_REFNIS_NL,DT_VLDT_START,DT_VLDT_END
4,2,20000,-,Provinz Brabant,Province de Brabant,Provincie Brabant,01/01/1970,31/12/1994
5,2,20001,02000,Provinz Flämisch-Brabant,Province du Brabant flamand,Provincie Vlaams-Brabant,01/01/1995,31/12/9999
6,2,20002,03000,Provinz Wallonisch-Brabant,Province du Brabant wallon,Provincie Waals-Brabant,01/01/1995,31/12/9999
17,3,21000,04000,Bezirk Brüssel-Hauptstadt,Arrondissement de Bruxelles-Capitale,Arrondissement Brussel-Hoofdstad,01/01/1970,31/12/9999
18,3,23000,20001,Bezirk Halle-Vilvoorde,Arrondissement de Hal-Vilvorde,Arrondissement Halle-Vilvoorde,01/01/1995,31/12/9999
...,...,...,...,...,...,...,...,...
580,4,25120,25000,Orp-Jauche,Orp-Jauche,Orp-Jauche,01/01/1977,31/12/9999
581,4,25121,25000,Ottignies-Louvain-la-Neuve,Ottignies-Louvain-la-Neuve,Ottignies-Louvain-la-Neuve,01/01/1977,31/12/9999
582,4,25122,25000,Ramillies,Ramillies,Ramillies,01/01/1977,31/12/9999
583,4,25123,25000,Rebecq,Rebecq,Rebecq,01/01/1977,31/12/9999


In [62]:
def get_nic_codes_for_regions(df):
    return df[df["LVL_REFNIS"] == 1]
    return df[(df["CD_REFNIS"].astype(str).str.startswith("2")) & (df["CD_REFNIS"].astype(str).str.len() == 5)]

In [63]:
def get_nic_codes_for_provinces(df):
    return df[df["LVL_REFNIS"] == 2]
    return df[(df["CD_REFNIS"].astype(str).str.startswith("2")) & (df["CD_REFNIS"].astype(str).str.len() == 5)]

In [66]:
def get_nic_codes_for_districts(df):
    return df[df["LVL_REFNIS"] == 3]
    return df[(df["CD_REFNIS"].astype(str).str.startswith("2")) & (df["CD_REFNIS"].astype(str).str.len() == 5)]

In [68]:
get_nic_codes_for_provinces(df)

,LVL_REFNIS,CD_REFNIS,CD_SUP_REFNIS,TX_REFNIS_DE,TX_REFNIS_FR,TX_REFNIS_NL,DT_VLDT_START,DT_VLDT_END,test
3,2,10000,02000,Provinz Antwerpen,Province d'Anvers,Provincie Antwerpen,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...
4,2,20000,-,Provinz Brabant,Province de Brabant,Provincie Brabant,01/01/1970,31/12/1994,\n <HelloWorld(name='Hello World')>...
5,2,20001,02000,Provinz Flämisch-Brabant,Province du Brabant flamand,Provincie Vlaams-Brabant,01/01/1995,31/12/9999,\n <HelloWorld(name='Hello World')>...
6,2,20002,03000,Provinz Wallonisch-Brabant,Province du Brabant wallon,Provincie Waals-Brabant,01/01/1995,31/12/9999,\n <HelloWorld(name='Hello World')>...
7,2,30000,02000,Provinz Westflandern,Province de Flandre occidentale,Provincie West-Vlaanderen,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...
8,2,40000,02000,Provinz Ostflandern,Province de Flandre orientale,Provincie Oost-Vlaanderen,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...
9,2,50000,03000,Provinz Hennegau,Province du Hainaut,Provincie Henegouwen,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...
10,2,60000,03000,Provinz Lüttich,Province de Liège,Provincie Luik,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...
11,2,70000,02000,Provinz Limburg,Province du Limbourg,Provincie Limburg,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...
12,2,80000,03000,Provinz Luxemburg,Province du Luxembourg,Provincie Luxemburg,01/01/1970,31/12/9999,\n <HelloWorld(name='Hello World')>...


In [56]:
class HelloWorld():
    def __init__(self):
        self.name = "Hello World"        
    def __repr__(self):
        return """
            <HelloWorld(name='%s')>
        """ % (self.name)

In [57]:
hello = HelloWorld()
hello


            <HelloWorld(name='Hello World')>
        

In [58]:
df['test'] = hello